In [1]:
import os
os.chdir('/home/blueconet/Downloads/Image Classification')

In [2]:
from cifar_input import cifar10_input
import ResNet_cifar10 as model
import tensorflow as tf

BATCH_SIZE=16
EVAL_SIZE=1000
NUM_GPUS=8

def tower_loss(batch, scope):
    images, labels = batch
    
    logits = model.inference(images, keep_prob)
    _ = model.loss(logits, labels)
    
    losses = tf.get_collection('losses', scope)
    
    total_loss = tf.add_n(losses, name='total_loss')
    return total_loss

def average_grads(list_grads):
    grads = [(tf.reduce_mean([list_grads[j][i][0] for j in range(len(list_grads))],
                             0),
             list_grads[0][i][1]) for i in range(len(list_grads[0]))]
    return grads

In [5]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
    train_data, test_data = cifar10_input()
    
    train_batch = train_data.dist_batch(BATCH_SIZE,20000)
    test_batch = test_data.batch(200)

    x = tf.placeholder(tf.float32, [None, 32, 32, 3])
    y_ = tf.placeholder(tf.int32, [x.get_shape()[0]])
    keep_prob = tf.placeholder(tf.float32)
    lr = tf.placeholder(tf.float32)
    opt = tf.train.GradientDescentOptimizer(lr)

    with tf.device('/gpu:0'):
        logit = model.inference(x, keep_prob)
        loss = model.loss(logit, y_)
        grads = opt.compute_gradients(loss)    
    
    train_op = opt.apply_gradients(grads)
    sess = tf.Session()
    
    rate = 0.01
    
    sess.run(tf.initialize_all_variables())
    tf.train.start_queue_runners(sess=sess)
    for i in range(20):
        image_batch, label_batch = sess.run(train_batch)
        _, cross_entropy = sess.run([train_op, loss], feed_dict = {x: image_batch, y_: label_batch, keep_prob: 0.5, lr:rate})
        print("step %d, cross_entropy %g"%(i, cross_entropy))

step 0, cross_entropy 4.64987
step 1, cross_entropy 8.16303
step 2, cross_entropy 3.81322
step 3, cross_entropy 3.13655
step 4, cross_entropy 2.31367
step 5, cross_entropy 2.70955
step 6, cross_entropy 2.27764
step 7, cross_entropy 2.42073
step 8, cross_entropy 2.30663
step 9, cross_entropy 2.22418
step 10, cross_entropy 2.18928
step 11, cross_entropy 2.46275
step 12, cross_entropy 2.0507
step 13, cross_entropy 2.28102
step 14, cross_entropy 2.0481
step 15, cross_entropy 2.49826
step 16, cross_entropy 2.20688
step 17, cross_entropy 2.15882
step 18, cross_entropy 2.33015
step 19, cross_entropy 2.00647


In [32]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)

        with tf.device('/gpu:0'):
            with tf.name_scope('%s_%d' % ('tower', 0)) as scope:
                loss = tower_loss(train_batch, scope)
                grads = opt.compute_gradients(loss)    
    
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.1
        mean_loss = 0.
        least = 10.
        count = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean_loss = mean_loss + cross_entropy/50
    
            if i%50 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, mean_loss))
      
                if mean_loss > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean_loss
                mean_loss=0
        
            if i%250 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                print("test accuracy %g"%(eval_accuracy))

step 50, cross_entropy 2.34179
step 100, cross_entropy 2.20467
step 150, cross_entropy 2.11237
step 200, cross_entropy 2.05938
step 250, cross_entropy 2.01351
test accuracy 0.236
step 300, cross_entropy 2.01619
step 350, cross_entropy 1.96637
step 400, cross_entropy 1.9585
step 450, cross_entropy 1.9671
step 500, cross_entropy 1.93296
test accuracy 0.236
step 550, cross_entropy 1.94518
step 600, cross_entropy 1.89222
step 650, cross_entropy 1.88125
step 700, cross_entropy 1.83801
step 750, cross_entropy 1.85326
test accuracy 0.317
step 800, cross_entropy 1.86021
step 850, cross_entropy 1.83193
step 900, cross_entropy 1.80373
step 950, cross_entropy 1.77622
step 1000, cross_entropy 1.78349
test accuracy 0.332
step 1050, cross_entropy 1.80523
step 1100, cross_entropy 1.79015
step 1150, cross_entropy 1.73958
step 1200, cross_entropy 1.72495
step 1250, cross_entropy 1.76043
test accuracy 0.333
step 1300, cross_entropy 1.75862
step 1350, cross_entropy 1.7238
step 1400, cross_entropy 1.73104

In [6]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)
        
        tower_grads = []
        tower_losses = []
        
        for i in range(NUM_GPUS):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                    loss = tower_loss(train_batch, scope)
                    grads = opt.compute_gradients(loss)
                    tower_losses.append(loss)
                    tower_grads.append(grads)
                    
        grads = average_grads(tower_grads)
        mean_loss = tf.reduce_mean(tower_losses)
  
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.01
        least = 10.
        mean = 0.
        count = 0.
        highest = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))
      
                if mean > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean
                
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
        print("highest accuracy: %g"%(highest))

step 199, cross_entropy 2.09473
test accuracy 0.223
step 399, cross_entropy 1.8579
test accuracy 0.297
step 599, cross_entropy 1.71028
test accuracy 0.339
step 799, cross_entropy 1.74742
test accuracy 0.335
step 999, cross_entropy 1.88075
test accuracy 0.336
step 1199, cross_entropy 1.66923
test accuracy 0.39
step 1399, cross_entropy 1.65223
test accuracy 0.379
step 1599, cross_entropy 1.77588
test accuracy 0.374
step 1799, cross_entropy 1.80815
test accuracy 0.43
step 1999, cross_entropy 1.41464
test accuracy 0.44
step 2199, cross_entropy 1.70094
test accuracy 0.383
step 2399, cross_entropy 1.40497
test accuracy 0.454
step 2599, cross_entropy 1.83029
test accuracy 0.401
step 2799, cross_entropy 1.36982
test accuracy 0.426
step 2999, cross_entropy 2.14853
test accuracy 0.412
step 3199, cross_entropy 1.43405
test accuracy 0.479
step 3399, cross_entropy 1.5532
test accuracy 0.452
step 3599, cross_entropy 1.38545
test accuracy 0.47
step 3799, cross_entropy 1.5421
test accuracy 0.462
step 

In [12]:
rate = 0.0002
with tf.device('/cpu:0'):
    for i in range(4000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 0.283338
test accuracy 0.774
step 399, cross_entropy 0.333769
test accuracy 0.802
step 599, cross_entropy 0.340824
test accuracy 0.783
step 799, cross_entropy 0.415238
test accuracy 0.794
step 999, cross_entropy 0.851695
test accuracy 0.795
step 1199, cross_entropy 0.444282
test accuracy 0.786
step 1399, cross_entropy 0.632221
test accuracy 0.798
step 1599, cross_entropy 0.443342
test accuracy 0.796
step 1799, cross_entropy 0.419323
test accuracy 0.788
step 1999, cross_entropy 0.737149
test accuracy 0.78
step 2199, cross_entropy 0.143549
test accuracy 0.748
step 2399, cross_entropy 0.456489
test accuracy 0.774
step 2599, cross_entropy 0.536486
test accuracy 0.794
step 2799, cross_entropy 0.15262
test accuracy 0.798
step 2999, cross_entropy 0.917369
test accuracy 0.783
step 3199, cross_entropy 0.356477
test accuracy 0.786
step 3399, cross_entropy 0.196373
test accuracy 0.769
step 3599, cross_entropy 0.414751
test accuracy 0.806
step 3799, cross_entropy 0.307421
t

In [7]:
rate

0.01